<a href="https://colab.research.google.com/github/naza-campioni/RAG-BIO/blob/main/RAG_BIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai faiss-cpu tiktoken nltk pypdf
# or if using sentence-transformers later
# !pip install sentence-transformers

In [ ]:
import json
import os
from pypdf import PdfReader

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import tiktoken
import openai
import faiss
import numpy as np

from openai import OpenAI
from itertools import chain

In [ ]:
def chunk_text(book, chunk_size=500, overlap=100):
    reader = PdfReader(book)
    chunks = []
    id = 1
    for i in range(len(reader.pages)):
      start = 0
      text = reader.pages[i].extract_text()
      text_length = len(text)
      while start < text_length:
          end = start + chunk_size
          chunk = text[start:end]
          chunks.append({"id": id, "source": book, "page": i+1, "text": chunk})
          start += chunk_size - overlap
          id += 1
    return chunks


In [ ]:
cc = chunk_text("/content/lowen__il-processo-terapeutico-in-analisi-bioenergetica.pdf")

In [ ]:
with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(cc, f, ensure_ascii=False, indent=2)

In [ ]:
with open('chunks.json', 'r') as file:
    chunks = json.load(file)

In [ ]:
os.environ["OPENAI_API_KEY"] = "[YOUR KEY HERE]"
openai.api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key="[YOUR KEY HERE]")

In [ ]:
# --- 1. Load some example text ---
text = """
Grounding is one of the central concepts in Bioenergetics.
To be grounded is to feel connected to the earth, to experience the flow of energy through the legs and feet.
Alexander Lowen emphasized grounding as the basis for emotional expression and stability.
"""

Sentences: ['\nGrounding is one of the central concepts in Bioenergetics.', 'To be grounded is to feel connected to the earth, to experience the flow of energy through the legs and feet.', 'Alexander Lowen emphasized grounding as the basis for emotional expression and stability.']


In [ ]:
# --- 2. Tokenize into sentences ---
sentences = []
for i in range(len(chunks)):
  sentences.append(sent_tokenize(chunks[i]['text']))
sentences = list(chain(*sentences))

print("Sentences:", sentences)

Sentences: ["Il processo dell'Analisi Bioenergetica\ndi Alexander Lowen\nLa terapia che Reich definisce vegetoterapia a carattere analitico implica l'analisi del modo di \ncomportamento tipico o caratterologico dell'individuo insieme all'attenzione per le funzioni \ninvolontarie o vegetative del corpo, in particolare per quella respiratoria.", 'Reich aveva notato che il soggetto nevrotico inibisce la completa e naturale attività respiratoria.', 'Egli credeva, secondo me giustamente, che questo atteggiamento di inibi', 'rale attività respiratoria.', 'Egli credeva, secondo me giustamente, che questo atteggiamento di inibizione fosse legato alla \npaura di abbandonarsi completamente al sentimento sessuale.', "Se un individuo respirasse \nliberamente e pienamente, vale a dire senza nessun freno consapevole, si produrrebbe un \nmovimento del corpo simile ad un'onda con il risultato di un movimento spontaneo del bacino in \navanti con l'espirazione, e uno corrispondente indietro con l'inspir

In [ ]:
# --- 3. Count tokens (optional, for chunking) ---
enc = tiktoken.encoding_for_model("gpt-5-nano")
for i, s in enumerate(sentences):
    print(f"Sentence {i}: {len(enc.encode(s))} tokens")

Sentence 0: 78 tokens
Sentence 1: 23 tokens
Sentence 2: 20 tokens
Sentence 3: 5 tokens
Sentence 4: 39 tokens
Sentence 5: 77 tokens
Sentence 6: 8 tokens
Sentence 7: 22 tokens
Sentence 8: 37 tokens
Sentence 9: 56 tokens
Sentence 10: 26 tokens
Sentence 11: 4 tokens
Sentence 12: 26 tokens
Sentence 13: 13 tokens
Sentence 14: 25 tokens
Sentence 15: 33 tokens
Sentence 16: 28 tokens
Sentence 17: 27 tokens
Sentence 18: 19 tokens
Sentence 19: 65 tokens
Sentence 20: 13 tokens
Sentence 21: 12 tokens
Sentence 22: 42 tokens
Sentence 23: 35 tokens
Sentence 24: 22 tokens
Sentence 25: 14 tokens
Sentence 26: 13 tokens
Sentence 27: 48 tokens
Sentence 28: 17 tokens
Sentence 29: 47 tokens
Sentence 30: 27 tokens
Sentence 31: 31 tokens
Sentence 32: 42 tokens
Sentence 33: 18 tokens
Sentence 34: 8 tokens
Sentence 35: 29 tokens
Sentence 36: 20 tokens
Sentence 37: 40 tokens
Sentence 38: 17 tokens
Sentence 39: 16 tokens
Sentence 40: 9 tokens
Sentence 41: 39 tokens
Sentence 42: 29 tokens
Sentence 43: 26 tokens
Sen

In [ ]:
for i,s in enumerate(sentences):
  print(i,s)
  print(enc.encode(s))

0 Il processo dell'Analisi Bioenergetica
di Alexander Lowen
La terapia che Reich definisce vegetoterapia a carattere analitico implica l'analisi del modo di 
comportamento tipico o caratterologico dell'individuo insieme all'attenzione per le funzioni 
involontarie o vegetative del corpo, in particolare per quella respiratoria.
[8438, 24211, 18693, 6, 19424, 4496, 29548, 944, 522, 1578, 198, 4091, 34569, 18166, 262, 198, 4579, 98074, 1378, 63120, 19263, 48541, 14412, 91419, 261, 83910, 68, 3758, 278, 2424, 79543, 305, 6, 15134, 4496, 1083, 18109, 1320, 793, 639, 447, 4626, 10511, 2424, 293, 83910, 188467, 18693, 82245, 17188, 18364, 100456, 722, 6, 27434, 23152, 777, 505, 2827, 63253, 793, 258, 2978, 645, 47275, 293, 14412, 1799, 1083, 31030, 11, 306, 105067, 777, 60468, 30508, 44930, 13]
1 Reich aveva notato che il soggetto nevrotico inibisce la completa e naturale attività respiratoria.
[720, 703, 94563, 625, 2754, 1378, 1793, 24712, 61978, 45151, 8150, 2424, 306, 526, 48541, 557, 379

In [ ]:
# --- 4. Generate embeddings ---
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return np.array(response.data[0].embedding, dtype=np.float32)

embeddings = [get_embedding(s) for s in sentences]
embeddings = np.vstack(embeddings)

In [ ]:
embeddings

array([[ 0.00596525,  0.03525065,  0.02678795, ...,  0.0114957 ,
         0.00545622,  0.02195212],
       [ 0.01993451,  0.00681176, -0.01212119, ...,  0.01590416,
        -0.02240822, -0.00891743],
       [ 0.02082571, -0.01136808,  0.00706958, ..., -0.00145884,
         0.01342984,  0.00710268],
       ...,
       [ 0.01477281, -0.00388182, -0.02560296, ..., -0.03404805,
         0.00021314,  0.0087493 ],
       [-0.00333373, -0.05243227, -0.01511028, ..., -0.00906419,
        -0.00972009, -0.00924666],
       [ 0.0549788 , -0.01786284,  0.03096507, ...,  0.01750474,
         0.01293371, -0.02115947]], dtype=float32)

In [ ]:
# --- 5. Store in FAISS index ---
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
print(f"Added {index.ntotal} vectors to FAISS")

Added 263 vectors to FAISS


In [ ]:
# --- 6. Search for a query ---
query = "Che cos'è la terapia bioenergetica?"
query_emb = get_embedding(query).reshape(1, -1)

In [ ]:
k = 5  # top 2 results
D, I = index.search(query_emb, k)
print("\nQuery:", query)
print("Top results:")
for rank, idx in enumerate(I[0]):
    print(f"{rank+1}. {sentences[idx]}")


Query: Che cos'è la terapia bioenergetica?
Top results:
1. Il processo dell'Analisi Bioenergetica
di Alexander Lowen
La terapia che Reich definisce vegetoterapia a carattere analitico implica l'analisi del modo di 
comportamento tipico o caratterologico dell'individuo insieme all'attenzione per le funzioni 
involontarie o vegetative del corpo, in particolare per quella respiratoria.
2.  terapeuti bioenergetici, sappiamo che la realtà della vita è nel corpo e che a questo livello in 
cui ci troviamo siano stati tutti rovinati dalla nostra educazione e dalla nostra cultura.
3. Noi, terapeuti bioenergetici, sappiamo che la realtà della vita è nel corpo e che a questo livello in 
c
4. La terapia è un viaggio alla scoperta di sé.
5. In realtà si tratta di un mio 
fallimento, non di un fallimento dell'Analisi Bioenergetica, che è un modo di capire una persona e i 
suoi problemi emozionali in termini di energia dinamic


In [ ]:
retrieved_text = list(sentences[i] for i in I.flatten())

In [ ]:
def generate_answer(query, retrieved_text):
    """
    query: the user question
    retrieved_text: string containing the most relevant chunk(s)
    """
    prompt = f"""
    You are a helpful librarian. Answer the following question using ONLY the information provided below.
    If the information is not in the context, say you don't know. Your personality is warm, grounded and secure.

    Context:
    {retrieved_text}

    Question:
    {query}

    Answer:
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

In [ ]:
generate_answer("Cos'è la terapia bioenergetica?",retrieved_text)

'La terapia bioenergetica, secondo il testo, è la terapia che Reich definisce vegetoterapia a carattere analitico: implica l’analisi del modo di comportamento tipico o caratterologico dell’individuo insieme all’attenzione per le funzioni involontarie o vegetative del corpo, in particolare per quella respiratoria. È descritta anche come un viaggio alla scoperta di sé e come un modo di capire i problemi emozionali di una persona in termini di energia dinamica.'